In [1]:
from tensorflow import keras


In [2]:
#Model_path = "C:/Users/uqcche32/OneDrive - The University of Queensland/PhD/HPC_Results/Sugarcane_disease/ML/Attention_Test"
Model_path = "E:/learning resource/OneDrive - The University of Queensland/PhD/HPC_Results/Sugarcane_disease/ML/"

In [3]:
from CustomLayers import *

In [4]:
model_folder = "test_model"
full_path = Model_path + "/" + model_folder
model = keras.models.load_model(full_path,custom_objects={"MultiHead_QKV_BlockAttention": MultiHead_QKV_BlockAttention,
                                                         "SNPBlockLayer":SNPBlockLayer})

In [5]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer_1 (InputLayer)      [(None, 100, 1)]     0                                            
__________________________________________________________________________________________________
zero_padding1d (ZeroPadding1D)  (None, 100, 1)       0           input_layer_1[0][0]              
__________________________________________________________________________________________________
locally_connected1d (LocallyCon (None, 10, 12)       1200        zero_padding1d[0][0]             
__________________________________________________________________________________________________
dense (Dense)                   (None, 10, 12)       156         locally_connected1d[0][0]        
_______________________________________________________________________________________

In [ ]:
!pip install shap
import shap

  Attempting uninstall: packaging
    Found existing installation: packaging 20.1
    Uninstalling packaging-20.1:
      Successfully uninstalled packaging-20.1


ERROR: pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
ERROR: pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
